In [1]:
!pip install -qU malariagen_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.9/775.9 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB

In [2]:
import allel
import malariagen_data
import itertools
import numpy as np
import re
import plotly.express as px
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
#pd.reset_option('^display.', silent=True)

In [3]:
import statsmodels as stats
from dask.diagnostics.progress import ProgressBar

In [ ]:
#Mounting Google Drive
#import os
#from google.colab import drive
#drive.mount("drive")

# make dir
#results_dir = '/home/mkient/gitrepos/IR/'
#os.makedirs(results_dir, exist_ok=True)
#os.chdir(results_dir)
#os.listdir()

In [4]:
#Mounting Google Drive
import os
from google.colab import drive
drive.mount("drive")

# make dir
results_dir = "drive/MyDrive"
os.makedirs(results_dir, exist_ok=True)

Mounted at drive


In [5]:
## Importing malariagen data set
ag3 = malariagen_data.Ag3("gs://vo_agam_release_master_us_central1/", pre=True)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release_master_us_central1/
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 3.10, 3.11, 3.12, 3.13, 3.14, 3.15, 3.16
Results cache           : None
Cohorts analysis        : 20250815
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 15.4.0
Client location         : Oregon, United States (Google Cloud us-west1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v15.4.0/Ag3.html

In [6]:
df_samples=ag3.sample_metadata(sample_sets='3.11', sample_query='country=="Burkina Faso"')
sets = list(df_samples.sample_set.unique())

## load haplotypes data
haplo_sample_id = np.load('drive/MyDrive/sanger_analyses/resistance/VGSC/data/haplo_sample_id_all.npy',
                          allow_pickle=True).item()

## mapping to df_samples
map_dict = {}
for hap in haplo_sample_id.keys():
  hap_i = haplo_sample_id[hap]
  for sample_id in hap_i:
    map_dict[sample_id] = hap
len(map_dict.keys())
df_samples['haplo_id'] = df_samples.sample_id.map(map_dict)

## building cohorts
popul_id = []
for loca, esp, hap, year in zip(df_samples.location, df_samples.aim_species, df_samples.haplo_id, df_samples.year):
  popul_id.append(loca[:4]+'_'+esp[:3]+'_'+hap+'_'+str(year))
df_samples['pop_haps'] = popul_id

cohort_haps, cohort_haps_size = {}, {}
for pop_haps in df_samples.pop_haps.unique():
  s_id = list(df_samples.query(f'pop_haps == "{pop_haps}"').sample_id)
  cohort_haps[pop_haps] = f'sample_id == {s_id}'
  cohort_haps_size[pop_haps] = len(s_id)

In [21]:
df_samples.query('year >2021').groupby(['aim_species','location']).size()

aim_species                       location      
arabiensis                        Gama               13
                                  Nagare              9
                                  Nassan             13
                                  Ouro-Hesso         10
                                  Po-Dongo           11
                                  Sideradougou        8
                                  Souroukoudinga      7
coluzzii                          Bana Village      120
                                  Degue-Degue         4
                                  Gama               39
                                  Nagare            100
                                  Nassan             32
                                  Ouro-Hesso         61
                                  Po-Dongo           20
                                  Sideradougou       10
                                  Souroukoudinga     61
gambiae                           Bana Village        3
                                  Degue-Degue         5
                                  Gama               27
                                  Nagare             13
                                  Nassan              2
                                  Po-Dongo           18
                                  Sideradougou       26
                                  Souroukoudinga     16
intermediate_gambcolu_arabiensis  Ouro-Hesso          1
intermediate_gambiae_coluzzii     Bana Village        1
                                  Degue-Degue         1
dtype: int64

In [7]:
haplo_sample_id.keys()

dict_keys(['FVI/FVI', 'FL1T/LVI', 'LL1T/LL1T', 'LL1T/LL2T', 'FL2T/LVI', 'OD'])

In [8]:
## define samples cohorts
cohorts, coh112, cohorts_, pop_list, pop_list1 = {},{}, {}, [],[]
for loca in df_samples.location.unique():
    loca_sample = df_samples.query(f"location == '{loca}'")
    for species in loca_sample.aim_species.unique():
      sp_sample=loca_sample.query(f'aim_species=="{species}"')
      key2 = loca[:4]+'_'+species[:3]
      key3 = loca+'[An. '+species+']'
      cohorts_[key2] = f"country == 'Burkina Faso' and location=='{loca}' and aim_species == '{species}'"
      pop_list1.append(key3)
      for year in sp_sample.year.unique():
        key = loca[:4]+'_'+species[:3]+'_'+str(year)
        key1 = loca+'[An. '+species+' ('+ str(year)+')]'
        cohorts[key] = f"country == 'Burkina Faso' and location=='{loca}' and aim_species == '{species}' and year == {year}"
        sample_list = list(df_samples.query(f"country == 'Burkina Faso' and location=='{loca}' and taxon == '{species}' and year == {year}").sample_id)
        coh112[key] = "sample_id in ['"+ "', '".join(sample_list) + "']"
        pop_list1.append(key1)

## Cohorts size
cohorts_size, cohorts_size_ = {}, {}
for coh in cohorts.keys():
  cohorts_size[coh] = df_samples.query(f'{cohorts[coh]}').shape[0]
for coh in cohorts_.keys():
  cohorts_size_[coh] = df_samples.query(f'{cohorts_[coh]}').shape[0]

taxons = {
    "col":"coluzzii",
    "gam":"gambiae",
    "ara":"arabiensis",
    "int":"intermediate_gambiae_coluzzii"
}
areas = {
    "Bana":"Bana Village",
    "Sour":"Souroukoudinga",
    "Nass":"Nassan",
    "Side":"Sideradougou",
    "Degu":"Degue-Degue",
    "Po-D":"Po-Dongo",
    "Gama":"Gama",
    "Ouro":"Ouro-Hesso",
    "Naga":"Nagare"
}

In [9]:
haplo_dict = np.load('drive/MyDrive/sanger_analyses/resistance/VGSC/data/haplo_dict.npy',
                     allow_pickle=True).item()
haplo_coh, haplo_coh_size = {}, {}
for hap in haplo_dict.keys():
  haplo_coh[hap] = f'sample_id == {haplo_dict[hap]}'
for hap in haplo_dict.keys():
  haplo_coh_size[hap] = len(haplo_coh[hap])

In [ ]:
#df_samples['haplo_id'] = df_samples.sample_id.map(map_dict)

In [10]:
def creat_dict(string_val):
  container = {}
  list_chain = string_val.split('_')
  for char in list_chain:
    container[f'key_{list_chain.index(char)}'] = []
  return container

def add_ci_method(count=28,nobs=46,method='normal'):
  from statsmodels.stats.proportion import proportion_confint
  ci_lower, ci_upper = proportion_confint(count=count,nobs=nobs,method=method)
  return ci_lower, ci_upper

def add_ci(p,n):
  sdt_err = np.sqrt((p*(1-p))/(n))
  return sdt_err

query="country == 'Burkina Faso' and location=='Bana Village' and aim_species == 'coluzzii' and year == 2021"

In [11]:
def creat_dict(string_val):
  container = {}
  list_chain = string_val.split('_')
  for char in list_chain:
    container[f'key_{list_chain.index(char)}'] = []
  return container

def ghost_table(data, cohorts_size, id_vars, taxons, areas):

  data_melt = data.melt(id_vars = id_vars, var_name = 'populations', value_name = 'frequency')

  coh_size = []
  for val in [np.repeat(cohorts_size[key.split('frq_')[1]],data.shape[0]) for key in data_melt.populations.unique()]:
    for i in val:
      coh_size.append(i)
  data_melt.insert(12,'coh_size',coh_size)
  col_dict = creat_dict(data_melt.populations.unique()[0])
  gost_keys = list(col_dict.keys())
  for val in data_melt.populations:
    gost_val = val.split('_')
    #if 'frq' in gost_val:
    #  gost_val.remove('frq')
    for val_i in gost_val:
      idx = gost_val.index(val_i)
      col_dict[gost_keys[idx]].append(val_i)
  for key in col_dict.keys():
    data_melt.insert(13,f'{key}',col_dict[key])
    data_melt_sel = data_melt.query('frequency>0')
  return data_melt_sel, data_melt

In [12]:
nobs = 2*df_samples.query("country == 'Burkina Faso' and location=='Bana Village' and aim_species == 'coluzzii' and year == 2021").shape[0]

In [ ]:
#np.repeat(nobs, 23).tolist()

In [13]:
def ds_vars(region='AGAP004707-RD',samples_sets='3.11', sample_query=query, pop='Bana_col_2021', filt='gamb_colu_arab'):
  ##
  ds_snps = ag3.snp_calls(region=region, sample_sets=samples_sets,sample_query=sample_query)
  s_pos = allel.SortedIndex(ds_snps['variant_position'].values)

  # To filter the SNP dataset and warp the dataset to GT array
  filt_val = ds_snps[f"variant_filter_pass_{filt}"].values
  ds_gt_filtered = allel.GenotypeArray(ds_snps["call_genotype"][filt_val].data)

  ## To count the number of alleles
  with ProgressBar():
      ds_ac = ds_gt_filtered.count_alleles(max_allele=3)

  ##
  ds_sites = ds_snps['variant_allele'][filt_val].compute()
  nobs = 2*ag3.sample_metadata(sample_sets=samples_sets, sample_query=sample_query).shape[0]

  ##
  is_var = ds_ac.is_variant()
  pos_filt = allel.SortedIndex(ds_snps['variant_position'][filt_val][is_var].compute())
  #ds_count = ds_gt_filtered[is_var].count_alleles()

  ##
  df_count = pd.DataFrame({'nobs':np.repeat(nobs, len(pos_filt)).tolist(), 'ref_count':ds_ac[is_var][:,0],
                           'alt1':ds_ac[is_var][:,1], 'alt2':ds_ac[is_var][:,2],'alt3':ds_ac[is_var][:,3]})
  df_sites = pd.DataFrame({'REF':ds_sites[is_var][:,0],'ALT1':ds_sites[is_var][:,1],'ALT2':ds_sites[is_var][:,2],
                           'ALT3':ds_sites[is_var][:,3]})

  df_count_melt = df_count.melt(id_vars = ['ref_count','nobs'], var_name = 'alt_idx', value_name = 'alt_count')
  df_sites_melt = df_sites.melt(id_vars = ['REF'], var_name = 'ALT_allele', value_name = 'SNP')

  ###
  REF = [ref_i.decode('ISO-8859-1') for ref_i in df_sites_melt.REF]
  ALT_allele = [alt.decode('ISO-8859-1') for alt in df_sites_melt.SNP]
  pos1 = list(pos_filt)

  ###
  df_count_melt.insert(0,'ref_allele',REF)
  df_count_melt.insert(1,'alt_allele',ALT_allele)
  df_count_melt.insert(0,'pos',pos1*3)

  ##
  col_dict = creat_dict(pop)
  gost_keys = list(col_dict.keys())
  gost_val = pop.split('_')
  for val_i in gost_val:
    idx = gost_val.index(val_i)
    col_dict[gost_keys[idx]] = np.repeat(val_i, len(df_count_melt.pos))
  for key in col_dict.keys():
    df_count_melt.insert(5,f'{key}',col_dict[key])
  df_vars = df_count_melt.query('alt_count>0').sort_values(by=['pos'], ascending=True)
  return df_vars, df_count_melt

In [14]:
id_vars = ['contig', 'position', 'ref_allele', 'alt_allele', 'aa_change',
           'pass_gamb_colu_arab', 'pass_gamb_colu', 'pass_arab','transcript',
           'effect', 'impact', 'ref_codon', 'alt_codon', 'aa_pos','ref_aa',
           'alt_aa', 'label','max_af']
#freq_ds = ghost_table(data=snps_rd_y.reset_index(), cohorts=cohorts, id_vars=id_vars, taxons=taxons, areas=areas)

In [15]:
transcript_vgsc='AGAP004707-RD'
transcript_ace1='AGAP001356-RA'

In [ ]:
df_list=[]
transcript_vgsc='AGAP004707-RD'
for coh in cohorts:
  if cohorts_size[coh]>=5:
    df_vars, df_count_allele = ds_vars(region=transcript_vgsc,sample_query=cohorts[coh],pop=coh)
    snp_freq = ag3.snp_allele_frequencies(transcript=transcript_vgsc, sample_sets='3.11',
                                          cohorts=cohorts,sample_query=cohorts[coh],
                                         min_cohort_size=5)
    freq_ds,_ = ghost_table(data=snp_freq.reset_index(), cohorts_size=cohorts_size,
                           id_vars=id_vars, taxons=taxons, areas=areas)

    freq_ds=freq_ds.query('pass_gamb_colu_arab ==True')
    freq_ds=freq_ds.sort_values(by=['position','ref_allele','alt_allele'], ascending=True)
    df_vars=df_vars.sort_values(by=['pos','ref_allele','alt_allele'], ascending=True)
    freq_ds.insert(4,'alt_count',list(df_vars.alt_count))
    freq_ds.insert(2,'ref_count',list(df_vars.ref_count))
    freq_ds.insert(2,'nobs',list(df_vars.nobs))
    freq_ds['freq_ci_lower'],_ = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
    _, freq_ds['freq_ci_upper'] = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
    freq_ds['freq_error'] = add_ci(p=freq_ds.frequency,n=freq_ds.nobs)
    df_list.append(freq_ds)

df_vgsc=pd.concat(df_list)

In [ ]:
df_list=[]
transcript_vgsc='AGAP004707-RD'
for coh in haplo_coh.keys():
  df_vars, df_count_allele = ds_vars(region=transcript_vgsc,sample_query=haplo_coh[coh],pop=coh)
  snp_freq = ag3.snp_allele_frequencies(transcript=transcript_vgsc, sample_sets='3.11',
                                        cohorts=haplo_coh,sample_query=haplo_coh[coh],
                                       min_cohort_size=5)
  freq_ds,_ = ghost_table(data=snp_freq.reset_index(), cohorts_size=haplo_coh_size,
                          id_vars=id_vars, taxons=taxons, areas=areas)

  freq_ds=freq_ds.query('pass_gamb_colu_arab ==True')
  freq_ds=freq_ds.sort_values(by=['position','ref_allele','alt_allele'], ascending=True)
  df_vars=df_vars.sort_values(by=['pos','ref_allele','alt_allele'], ascending=True)
  freq_ds.insert(4,'alt_count',list(df_vars.alt_count))
  freq_ds.insert(2,'ref_count',list(df_vars.ref_count))
  freq_ds.insert(2,'nobs',list(df_vars.nobs))
  freq_ds['freq_ci_lower'],_ = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
  _, freq_ds['freq_ci_upper'] = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
  freq_ds['freq_error'] = add_ci(p=freq_ds.frequency,n=freq_ds.nobs)
  df_list.append(freq_ds)

df_haplo=pd.concat(df_list)

In [ ]:
df_list=[]
transcript_vgsc='AGAP004707-RD'
for coh in cohort_haps.keys():
  if cohort_haps_size[coh]>=5:
    df_vars, df_count_allele = ds_vars(region=transcript_vgsc,sample_query=cohort_haps[coh],pop=coh)
    snp_freq = ag3.snp_allele_frequencies(transcript=transcript_vgsc, sample_sets='3.11',
                                          cohorts=cohort_haps,sample_query=cohort_haps[coh],
                                          min_cohort_size=5)
    freq_ds,_ = ghost_table(data=snp_freq.reset_index(), cohorts_size=cohort_haps_size,
                            id_vars=id_vars, taxons=taxons, areas=areas)

    freq_ds=freq_ds.query('pass_gamb_colu_arab ==True')
    freq_ds=freq_ds.sort_values(by=['position','ref_allele','alt_allele'], ascending=True)
    df_vars=df_vars.sort_values(by=['pos','ref_allele','alt_allele'], ascending=True)
    freq_ds.insert(4,'alt_count',list(df_vars.alt_count))
    freq_ds.insert(2,'ref_count',list(df_vars.ref_count))
    freq_ds.insert(2,'nobs',list(df_vars.nobs))
    freq_ds['freq_ci_lower'],_ = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
    _, freq_ds['freq_ci_upper'] = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
    freq_ds['freq_error'] = add_ci(p=freq_ds.frequency,n=freq_ds.nobs)
    df_list.append(freq_ds)

df_haps=pd.concat(df_list)

In [ ]:
df_haplo.to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_haplo.csv')
df_haps.to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_haps.csv')

In [ ]:
#df_haps.head()

In [ ]:
df_vgsc_y = df_vgsc.copy()
df_vgsc_y['freq_error_'] = df_vgsc.frequency-df_vgsc.freq_ci_lower
df_vgsc_y['freq_error_minus'] = df_vgsc.freq_ci_upper-df_vgsc.frequency
df_vgsc_y.rename(columns={'key_1':'location','key_2':'species','key_3':'periods'}, inplace=True)

In [ ]:
#df_vgsc_y.to_csv('drive/MyDrive/sanger_analyses/resistance/df_vgsc_y.csv')
df_query1 = df_vgsc_y.query('aa_change==["L995F","L995S", "V402L", "I1527T", "N1570Y"]')
df_query1.columns

Index(['contig', 'position', 'nobs', 'ref_count', 'ref_allele', 'alt_allele',
       'alt_count', 'aa_change', 'pass_gamb_colu_arab', 'pass_gamb_colu',
       'pass_arab', 'transcript', 'effect', 'impact', 'ref_codon', 'coh_size',
       'periods', 'species', 'location', 'key_0', 'alt_codon', 'aa_pos',
       'ref_aa', 'alt_aa', 'label', 'max_af', 'populations', 'frequency',
       'freq_ci_lower', 'freq_ci_upper', 'freq_error', 'freq_error_',
       'freq_error_minus'],
      dtype='object')

In [ ]:
df_list1=[]
transcript_ace1='AGAP001356-RA'
for coh in cohorts:
  if cohorts_size[coh]>=5:
    df_vars, df_count_allele = ds_vars(region=transcript_ace1,sample_query=cohorts[coh],pop=coh)
    snp_freq = ag3.snp_allele_frequencies(transcript=transcript_ace1, sample_sets='3.11',
                                          cohorts=cohorts,sample_query=cohorts[coh],
                                         min_cohort_size=5)
    freq_ds,_ = ghost_table(data=snp_freq.reset_index(), cohorts_size=cohorts_size,
                           id_vars=id_vars, taxons=taxons, areas=areas)

    freq_ds=freq_ds.query('pass_gamb_colu_arab ==True')
    freq_ds=freq_ds.sort_values(by=['position','ref_allele','alt_allele'], ascending=True)
    df_vars=df_vars.sort_values(by=['pos','ref_allele','alt_allele'], ascending=True)
    freq_ds.insert(4,'alt_count',list(df_vars.alt_count))
    freq_ds.insert(2,'ref_count',list(df_vars.ref_count))
    freq_ds.insert(2,'nobs',list(df_vars.nobs))
    freq_ds['freq_ci_lower'],_ = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
    _, freq_ds['freq_ci_upper'] = add_ci_method(count=freq_ds.alt_count,nobs=freq_ds.nobs, method='normal')
    freq_ds['freq_error'] = add_ci(p=freq_ds.frequency,n=freq_ds.nobs)
    df_list1.append(freq_ds)

df_ace1=pd.concat(df_list1)

In [ ]:
df_ace1_y = df_ace1.copy()
df_ace1_y['freq_error_'] = df_ace1.frequency-df_ace1.freq_ci_lower
df_ace1_y['freq_error_minus'] = df_ace1.freq_ci_upper-df_ace1.frequency
df_ace1_y.rename(columns={'key_1':'location','key_2':'species','key_3':'periods'}, inplace=True)

In [ ]:
df_ace1_y.to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_ace1_y.csv')
df_query2 = df_ace1_y.query('aa_change=="G280S"')
#df_query2#.columns

In [ ]:
## Compute SNP allele frequencies by collection site and species
#help(ag3.snp_allele_frequencies)
rd = 'AGAP004707-RD'
snps_rd = ag3.snp_allele_frequencies(transcript=rd, cohorts=cohorts_, site_mask='gamb_colu_arab',
                                     sample_sets='3.11', drop_invariant=True, effects=True,
                                     min_cohort_size=5)

##
snps_rd_y = ag3.snp_allele_frequencies(transcript=rd, cohorts=cohorts, site_mask='gamb_colu_arab',
                                       sample_sets='3.11', drop_invariant=True, effects=True,
                                       min_cohort_size=5)
#print('done !')

Load SNP genotypes:   0%|          | 0/101 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/21 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/5425 [00:00<?, ?it/s]

Load SNP genotypes:   0%|          | 0/101 [00:00<?, ?it/s]

Compute allele frequencies:   0%|          | 0/23 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/5096 [00:00<?, ?it/s]

In [ ]:
snps_rd.reset_index().to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_vgsc_pop.csv')
snps_rd_y.reset_index().to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_vgsc_year.csv')

In [ ]:
## Compute SNP allele frequencies by collection site and species
#help(ag3.snp_allele_frequencies)
ra = 'AGAP001356-RA'
snps_ra = ag3.snp_allele_frequencies(transcript=ra, cohorts=cohorts_, site_mask='gamb_colu_arab',
                                     sample_sets='3.11', drop_invariant=True, effects=True,
                                     min_cohort_size=5)
snps_ra_y = ag3.snp_allele_frequencies(transcript=ra, cohorts=cohorts, site_mask='gamb_colu_arab',
                                       sample_sets='3.11', drop_invariant=True, effects=True,
                                       min_cohort_size=5)
#print('done !')

In [ ]:
snps_ra.reset_index().to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_ace1_pop.csv')
snps_ra_y.reset_index().to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_ace1_year.csv')

## Data handeling

In [ ]:
def plot_freq_time_series(data, height=450, width=900, template='plotly_white', xlab='Years', var_title='Genetic variants',
                          ylab= 'Allelic frequencies', color=None, title='AGAP004707-RD (Vgsc/para) SNP frequencies'):

  #plot time series frequencies
  fig = px.line(data,x="location", y="frequency", error_y="freq_error_plus",
                error_y_minus="freq_error_minus", color="label", markers=True,
                height=height,width=width,template=template, title=title,
                labels={"date": f'{xlab}', "frequency": f'{ylab}', "variant":f'{var_title}'})

  # figure layout
  fig.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
  fig.update_yaxes(range=[0, 1.0], ticks="outside", col=1)
  fig.update_xaxes(ticks="outside", col=1)

  return fig

In [ ]:
fig = px.bar(data_frame=df_query1,height=450, width=1500, template='plotly_white',x='populations',y="frequency", error_y="freq_error_",
             error_y_minus="freq_error_minus", color='label', barmode='group')
fig.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
fig.update_yaxes(range=[0, 1.0], ticks="outside", col=1)


In [ ]:
fig1 = px.bar(data_frame=df_query2,height=450, width=800, template='plotly_white',x='populations',y="frequency", error_y="freq_error_",
              error_y_minus="freq_error_minus", color='label', barmode='group')
fig1.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
fig1.update_yaxes(range=[0, 1.0], ticks="outside", col=1)

## Allelic frequencies

In [ ]:
def ds_freq_tab(ds):
  #extract cohorts into a dataframe
  cohort_vars = [v for v in ds if v.startswith("cohort_")]
  df_cohorts = ds[cohort_vars].to_dataframe()
  df_cohorts.columns = [c.split("cohort_")[1] for c in df_cohorts.columns]

  variant_labels = ds["variant_label"].values
  dfs = []
  for cohort_index, cohort in enumerate(df_cohorts.itertuples()):
    ds_cohort = ds.isel(cohorts=cohort_index)
    dict_df =  {"taxon": cohort.taxon, "area": cohort.area, "date": cohort.period_start, "period": str(cohort.period),
                "sample_size": cohort.size,"variant": variant_labels, "count": ds_cohort["event_count"].values,"nobs": ds_cohort["event_nobs"].values,
                "frequency": ds_cohort["event_frequency"].values, "frequency_ci_low": ds_cohort["event_frequency_ci_low"].values,
                "frequency_ci_upp": ds_cohort["event_frequency_ci_upp"].values
                }
    df = pd.DataFrame(dict_df)
    dfs.append(df)

  df_events = pd.concat(dfs, axis=0).reset_index(drop=True)
  df_events = df_events.query("nobs > 0")

  # Frequencies stats
  frq = df_events["frequency"]
  frq_ci_low = df_events["frequency_ci_low"]
  frq_ci_upp = df_events["frequency_ci_upp"]
  df_events["frequency_error"] = frq_ci_upp - frq
  df_events["frequency_error_minus"] = frq - frq_ci_low

  return df_events

In [ ]:
ds_aafreq_vgsc = ag3.aa_allele_frequencies_advanced(transcript=transcript_vgsc, area_by="location", period_by="year",
                                                    sample_sets='3.11', sample_query="country == 'Burkina Faso'")
df_events_vgsc = ds_freq_tab(ds_aafreq_vgsc)
df_events_vgsc['populations']=[str(x)[:4]+'_'+str(y)[:4]+'_'+str(z) for x,y,z in zip(df_events_vgsc.area,df_events_vgsc.taxon,df_events_vgsc.period)]
df_events_vgsc.to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_events_vgsc.csv')

Load SNP genotypes:   0%|          | 0/139 [00:00<?, ?it/s]

Compute SNP allele frequencies:   0%|          | 0/18 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/19963 [00:00<?, ?it/s]

In [ ]:
ds_aafreq_ace1 = ag3.aa_allele_frequencies_advanced(transcript=transcript_ace1, area_by="location", period_by="year",
                                                    sample_sets='3.11', sample_query="country == 'Burkina Faso'")
df_events_ace1 = ds_freq_tab(ds_aafreq_ace1)
df_events_ace1['populations']=[str(x)[:4]+'_'+str(y)[:4]+'_'+str(z) for x,y,z in zip(df_events_ace1.area,df_events_ace1.taxon,df_events_ace1.period)]
df_events_ace1.to_csv('drive/MyDrive/sanger_analyses/resistance/data/df_events_ace1.csv')

Load SNP genotypes:   0%|          | 0/70 [00:00<?, ?it/s]

Compute SNP allele frequencies:   0%|          | 0/18 [00:00<?, ?it/s]

Compute SNP effects:   0%|          | 0/7546 [00:00<?, ?it/s]

In [ ]:
var = ['V402L (2L:2,391,228 G>{C,T})','I1527T (2L:2,429,617 T>C)','L995F (2L:2,422,652 A>T)','L995S (2L:2,422,651 T>C)','N1570Y (2L:2,429,745 A>T)']

In [ ]:
df_query3 = df_events_vgsc.query(f'variant=={var}')
#df_query3

In [ ]:
fig2 = px.bar(data_frame=df_query3.sort_values(by='populations',ascending=True),height=450, width=1500, template='plotly_white',x='populations',y="frequency", error_y="frequency_error",
             error_y_minus="frequency_error_minus", color='variant', barmode='group')
fig2.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
fig2.update_yaxes(range=[0, 1.0], ticks="outside", col=1)


In [ ]:
fig = px.bar(data_frame=df_query1.sort_values(by='populations',ascending=True),height=450, width=1500, template='plotly_white',x='populations',y="frequency", error_y="freq_error_",
             error_y_minus="freq_error_minus", color='label', barmode='group')
fig.update_layout(xaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='black'),
                   yaxis=dict(showgrid=False, showline=True, linewidth=1, linecolor='gray'))
fig.update_yaxes(range=[0, 1.0], ticks="outside", col=1)


In [ ]:
df_samples.columns

Index(['sample_id', 'partner_sample_id', 'contributor', 'country', 'location',
       'year', 'month', 'latitude', 'longitude', 'sex_call', 'sample_set',
       'release', 'quarter', 'study_id', 'study_url',
       'terms_of_use_expiry_date', 'terms_of_use_url', 'unrestricted_use',
       'mean_cov', 'median_cov', 'modal_cov', 'mean_cov_2L', 'median_cov_2L',
       'mode_cov_2L', 'mean_cov_2R', 'median_cov_2R', 'mode_cov_2R',
       'mean_cov_3L', 'median_cov_3L', 'mode_cov_3L', 'mean_cov_3R',
       'median_cov_3R', 'mode_cov_3R', 'mean_cov_X', 'median_cov_X',
       'mode_cov_X', 'frac_gen_cov', 'divergence', 'contam_pct', 'contam_LLR',
       'aim_species_fraction_arab', 'aim_species_fraction_colu',
       'aim_species_fraction_colu_no2l', 'aim_species_gambcolu_arabiensis',
       'aim_species_gambiae_coluzzii', 'aim_species', 'country_iso',
       'admin1_name', 'admin1_iso', 'admin2_name', 'taxon',
       'cohort_admin1_year', 'cohort_admin1_month', 'cohort_admin1_quarter',
      